In [1]:
import multiprocessing
import random
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
from deap import algorithms, base, creator, tools

%matplotlib notebook

## Problem and Solution Definitions

### Class that holds the problem definition.

This class must implements:
* A method to represent the problem graphically (`__str__`)
* A method to generate a problem instance from an input file (`from_file`).

In [8]:
class Problem:
    def __init__(self, rows: int, cols: int, fleet: int, rides: List[List[int]], bonus: int, steps: int):
        self.rows = rows
        self.cols = cols
        self.fleet = fleet
        self.rides = rides
        self.bonus = bonus
        self.steps = steps

    def distance(self, x: Tuple[int, int], y: Tuple[int, int]) -> int:
        return abs(x[0] - y[0]) + abs(x[1] - y[1])
    
    @property
    def max_bound(self):
        if not hasattr(self, '_max_bound'):
            self._max_bound = len(self.rides) * \
            self.bonus + np.sum(np.apply_along_axis(lambda x: self.distance(x=(x[0], x[1]), y=(x[2], x[3])), 1, self.rides[:, 0:4]))
        
        return self._max_bound
    
    @property
    def rides_by_starting(self) -> List[List[int]]:
        return [k for k, ride in sorted(enumerate(self.rides), key=lambda x: x[1][4])]
    
    @classmethod
    def from_file(cls, file_path, *args, **kwargs):
        with open(file_path) as f:
            rows, cols, fleet, num_rides, bonus, steps = [int(i) for i in f.readline().split()]
            rides = np.array([[int(x) for x in line.strip().split()] for i, line in zip(range(num_rides), f.readlines())])

        return cls(rows, cols, fleet, rides, bonus, steps)

    def __str__(self):
        return '\n'.join([f'Ride(x={i[0], i[1]}, y={i[2], i[3]}, starting={i[4]}, finish={i[5]})' for i in self.rides])

    def __repr__(self):
        s = f'Problem{{rows={self.rows}, cols={self.cols}, fleet={self.fleet}, bonus={self.bonus}, steps={self.steps}}}'
        if len(self.rides) < 20:
            s += f'\n{str(self)}'
        return s

### Problem instance

In [21]:
problem = Problem.from_file('input/a_example.in')
problem

Problem{rows=3, cols=4, fleet=2, bonus=2, steps=10}
Ride(x=(0, 0), y=(1, 3), starting=2, finish=9)
Ride(x=(1, 2), y=(1, 0), starting=0, finish=9)
Ride(x=(2, 0), y=(2, 2), starting=0, finish=9)

### Class that contains a Solution

The solution to this problem will be represented by a class that can be written and represented, so that following methods must be implemented:
* Serialize the solution into a string (`__str__`).
* Create a string that represents the solution (`__repr__`).
* Write to file the solution (`to_file`).

In [45]:
class Solution:
    """
    A solution for this problem that consists of a structure [[r1, r2, ..., rn]...]
    """
    def __init__(self, data, problem):
        self.data = data
        self.problem = problem

    def to_file(self, file_path):
        with open(file_path, 'w') as f:
            f.write(self.__str__())

    def __str__(self):
        return '\n'.join(' '.join([len(s)] + [str(x) for x in s]) for s in self.data)

    def __repr__(self):
        return self.__str__()

## Genetic Algorithm's Operators

In order to generate a population for the Genetic Algorithm it's necessary to define following constants:
* `INDIVIDUAL_TYPE`: The base type for an individual.
* `INDIVIDUAL_SIZE`: Initial size of the individual.

Running the algorithm consists of generating and evaluating individuals, so it's required to define functions for:
* Generating a new individual based on problem instance (`generate`).
* Evaluating the current fitness value of the individual (`evaluate`).

If it's necessary, following functions can be defined:
* Mutate individuals (`mutate`).
* Crossover (`crossover`).

### Helpers

In [94]:
def distance(a, b, x, y):
    return abs(x - a) + abs(y - b)

def ride_length(ride: List[int]) -> int:
    return distance(*ride[0:4])

def can_reach(x: int, y: int, step: int, ride: List[int]) -> bool:
    print(ride, distance(x, y, ride[0], ride[1]) + step <= ride[5] - ride_length(ride))
    return distance(x, y, ride[0], ride[1]) + step <= ride[5] - ride_length(ride)

def find_itinerary(car: int, problem: Problem, rides: List[int]) -> List[int]:
    itinerary = []
    step = 0
    x, y = (0, 0)
    print(car)
    try:
        while step <= problem.steps:
            valid_ride_index = next(i for i in rides if can_reach(x, y, step, problem.rides[i]))
            rides.pop(valid_ride_index)
            valid_ride = problem.rides[valid_ride_index]
            x, y = valid_ride[2:4]
            itinerary.append(valid_ride_index)
            step += ride_length(valid_ride)
    except StopIteration:
        pass
    
    return itinerary

# HEURISTIC = problem.cols * problem.rows // problem.cells

# def learning_rate(x):
#     return int(max((HEURISTIC - x) // HEURISTIC, 0))

### Constants

In [6]:
INDIVIDUAL_SIZE = 500
INDIVIDUAL_TYPE = np.ndarray

### Generation

In [95]:
def generate(problem: Problem, size: int) -> Tuple[int]:
    result = []
    for _ in range(size):
        valid = False
        while not valid:
            s = generate_slice(problem)
            valid = is_individual_valid(result + [s], problem)
        result.append(s)

    return creator.Individual(result)

def generate(problem: Problem) -> Dict[int, List[int]]:
    solution = {}
    rides_completed = []
    for c in range(problem.fleet):
        rides_available = [i for i in problem.rides_by_starting if i not in rides_completed]
        rides = find_itinerary(c, problem, rides_available)
        solution[c] = rides
        rides_completed += rides

    return solution

In [96]:
generate(problem)

0
[1 2 1 0 0 9] True
[1 2 1 0 0 9] True
[1 2 1 0 0 9] True


IndexError: pop index out of range

### Evaluation

In [8]:
def evaluate(individual, problem: Problem) -> Tuple[float]:
    if individual.any() and is_individual_valid(individual, problem):
        return float(np.sum(np.apply_along_axis(slice_area, 1, individual))),
    else:
        return 0.0,

### Mutation

In [9]:
def mutate(individual, problem) -> Tuple:
    length = len(individual)
    # mutation = np.delete(individual, random.sample(range(length), random.randint(0, int(length*0.1))), axis=0)
    new_slices = max(1, int(learning_rate(length) * length)) * 5
    mutation = np.append(individual, [generate_slice(problem) for _ in range(new_slices)], axis=0)
    
    return creator.Individual(mutation),

### Crossover

In [10]:
def crossover_two_points(ind1, ind2) -> Tuple:
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

def crossover(ind1, ind2) -> Tuple:
    new_slices = max(1, int(learning_rate(len(ind1)) * len(ind1)))
    cx_point = random.randint(0, len(ind1) - new_slices)
    cx1 = np.append(ind1, ind2[cx_point:cx_point+new_slices], axis=0)
    cx2 = np.append(ind2, ind1[cx_point:cx_point+new_slices], axis=0)
    return creator.Individual(cx1), creator.Individual(cx2)

## DEAP toolbox

### Types

In [11]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", INDIVIDUAL_TYPE, fitness=creator.Fitness)

### Population

In [12]:
toolbox = base.Toolbox()
toolbox.register("individual", generate, size=INDIVIDUAL_SIZE, problem=problem)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### Genetic functions

In [13]:
toolbox.register("evaluate", evaluate, problem=problem)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate, problem=problem)
toolbox.register("select", tools.selTournament, tournsize=3)

### Multiprocessing

In [14]:
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

## Solution

### Hyperparameters

In [15]:
GENERATIONS = 2000
MU = 10
LAMBDA = 20
CROSSOVER_PROBABILITY = 0.6
MUTATION_PROBABILITY = 0.3

### Run the algorithm

In [ ]:
population = toolbox.population(n=MU)

hof = tools.HallOfFame(1, similar=np.array_equal)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

# population, log = algorithms.eaSimple(population, toolbox, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY, GENERATIONS, stats, halloffame=hof)
population, log = algorithms.eaMuPlusLambda(population, toolbox, MU, LAMBDA, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY, GENERATIONS, stats, halloffame=hof)
# population, log = algorithms.eaMuCommaLambda(population, toolbox, MU, LAMBDA, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY, GENERATIONS, stats, halloffame=hof)

gen	nevals	avg     	std          	min    	max    
0  	10    	[4237.7]	[66.52225192]	[4164.]	[4402.]
1  	17    	[4360.3]	[75.80112136]	[4253.]	[4435.]
2  	15    	[4448.2]	[17.98221343]	[4429.]	[4474.]
3  	20    	[4486.4]	[11.892855]  	[4472.]	[4508.]
4  	18    	[4512.7]	[17.42440817]	[4488.]	[4548.]
5  	20    	[4553.4]	[18.97472002]	[4528.]	[4597.]
6  	19    	[4588.3]	[25.05613697]	[4538.]	[4640.]
7  	16    	[4602.1]	[16.56773974]	[4583.]	[4640.]
8  	18    	[4634.8]	[18.91983087]	[4601.]	[4662.]
9  	17    	[4675.1]	[21.90182641]	[4640.]	[4707.]
10 	19    	[4722.5]	[17.73273809]	[4700.]	[4753.]
11 	16    	[4772.3]	[18.91057905]	[4750.]	[4801.]
12 	17    	[4811.6]	[12.75303885]	[4797.]	[4844.]
13 	17    	[4848.4]	[23.89644325]	[4816.]	[4877.]
14 	20    	[4901.5]	[17.76654159]	[4877.]	[4924.]
15 	19    	[4931.7]	[9.45568612] 	[4918.]	[4954.]
16 	19    	[4958.1]	[17.82386041]	[4936.]	[4989.]
17 	18    	[4999.2]	[20.12361796]	[4961.]	[5035.]
18 	19    	[5040.1]	[23.91840296]	[4998.]	[5078.]


163	18    	[9690.9]	[21.17758249]	[9669.]	[9718.]
164	19    	[9720.7]	[17.17003203]	[9688.]	[9762.]
165	17    	[9759.2]	[13.44470156]	[9729.]	[9774.]
166	20    	[9775.5]	[14.16509795]	[9756.]	[9803.]
167	19    	[9804.9]	[23.9726928] 	[9773.]	[9833.]
168	16    	[9844.3]	[22.71585349]	[9791.]	[9874.]
169	18    	[9881.2]	[17.4859944] 	[9844.]	[9919.]
170	20    	[9927.5]	[28.79322837]	[9886.]	[9970.]
171	19    	[9979.5]	[28.23561581]	[9920.]	[10013.]
172	18    	[10028.2]	[19.2239434] 	[10012.]	[10054.]
173	19    	[10061.8]	[11.68588893]	[10050.]	[10095.]
174	20    	[10085.5]	[17.20610357]	[10065.]	[10105.]
175	19    	[10118.5]	[22.30358716]	[10079.]	[10155.]
176	17    	[10145.] 	[17.01763791]	[10117.]	[10160.]
177	18    	[10179.6]	[18.18350901]	[10160.]	[10212.]
178	19    	[10202.6]	[21.37849387]	[10179.]	[10253.]
179	20    	[10245.8]	[8.6]        	[10225.]	[10259.]
180	19    	[10278.6]	[15.29182788]	[10246.]	[10293.]
181	15    	[10307.8]	[21.3157219] 	[10284.]	[10342.]
182	18    	[10346.7

319	16    	[14853.4]	[26.96738771]	[14796.]	[14887.]
320	17    	[14870.5]	[15.92011306]	[14848.]	[14889.]
321	19    	[14910.7]	[19.78913844]	[14884.]	[14937.]
322	18    	[14944.8]	[22.94253691]	[14904.]	[14977.]
323	17    	[14972.] 	[19.03680645]	[14941.]	[14989.]
324	20    	[14998.1]	[12.02039933]	[14977.]	[15027.]
325	20    	[15028.9]	[15.42368309]	[15000.]	[15048.]
326	19    	[15059.1]	[17.59801125]	[15037.]	[15085.]
327	18    	[15089.7]	[18.84170905]	[15060.]	[15124.]
328	20    	[15127.] 	[22.80789337]	[15086.]	[15162.]
329	19    	[15176.] 	[22.93468988]	[15142.]	[15218.]
330	19    	[15210.6]	[17.32166274]	[15169.]	[15238.]
331	17    	[15245.9]	[12.42135258]	[15223.]	[15263.]
332	17    	[15269.8]	[15.23679756]	[15242.]	[15297.]
333	18    	[15308.2]	[20.36565737]	[15276.]	[15354.]
334	18    	[15350.5]	[22.82213837]	[15315.]	[15393.]
335	19    	[15370.3]	[11.76477794]	[15361.]	[15404.]
336	19    	[15404.5]	[28.27100989]	[15370.]	[15462.]
337	20    	[15439.3]	[25.16366428]	[15412.]	[1

474	19    	[19944.7]	[18.77791256]	[19914.]	[19976.]
475	20    	[19974.9]	[12.37295438]	[19942.]	[19993.]
476	19    	[20007.1]	[16.33676835]	[19984.]	[20026.]
477	17    	[20027.8]	[17.19767426]	[19999.]	[20059.]
478	16    	[20047.9]	[12.44548111]	[20035.]	[20078.]
479	18    	[20078.9]	[17.5296891] 	[20048.]	[20098.]
480	18    	[20117.8]	[11.2409964] 	[20102.]	[20141.]
481	16    	[20143.7]	[20.5915031] 	[20114.]	[20178.]
482	16    	[20184.3]	[14.10709042]	[20161.]	[20203.]
483	19    	[20220.6]	[16.63850955]	[20196.]	[20246.]
484	18    	[20254.7]	[23.07401135]	[20222.]	[20295.]
485	18    	[20302.8]	[22.04903626]	[20262.]	[20340.]
486	19    	[20345.5]	[24.91284809]	[20307.]	[20385.]
487	20    	[20401.2]	[23.61694307]	[20360.]	[20441.]
488	18    	[20425.1]	[13.49407277]	[20404.]	[20441.]
489	20    	[20465.9]	[20.47168777]	[20441.]	[20491.]
490	17    	[20493.5]	[18.58090418]	[20449.]	[20517.]
491	18    	[20523.5]	[19.72435043]	[20491.]	[20554.]
492	18    	[20555.2]	[13.34765897]	[20543.]	[2

629	20    	[25296.9]	[18.21784839]	[25281.]	[25336.]
630	16    	[25323.9]	[15.15552704]	[25295.]	[25345.]
631	18    	[25368.2]	[25.72469631]	[25327.]	[25397.]
632	13    	[25410.4]	[32.45057781]	[25349.]	[25442.]
633	16    	[25453.6]	[18.36954001]	[25442.]	[25492.]
634	20    	[25497.7]	[20.97164753]	[25454.]	[25531.]
635	19    	[25542.7]	[32.09065284]	[25501.]	[25585.]
636	15    	[25568.8]	[21.76602858]	[25536.]	[25591.]
637	18    	[25584.] 	[16.69131511]	[25545.]	[25600.]
638	19    	[25612.5]	[11.60387866]	[25597.]	[25634.]
639	19    	[25641.1]	[20.98785363]	[25603.]	[25672.]
640	17    	[25672.6]	[11.07429456]	[25664.]	[25703.]
641	20    	[25706.2]	[19.16663768]	[25675.]	[25724.]
642	17    	[25729.1]	[7.87972081] 	[25715.]	[25736.]
643	20    	[25773.9]	[15.21479543]	[25748.]	[25790.]
644	19    	[25808.1]	[19.84162292]	[25784.]	[25839.]
645	19    	[25843.1]	[24.77276731]	[25803.]	[25887.]
646	17    	[25853.7]	[13.89280389]	[25837.]	[25887.]
647	17    	[25881.3]	[21.26993183]	[25861.]	[2

...

1092	17    	[40444.5]	[15.31176019]	[40423.]	[40462.]
1093	17    	[40470.6]	[24.99279896]	[40429.]	[40515.]
1094	17    	[40510.] 	[19.57038579]	[40473.]	[40527.]
1095	19    	[40536.5]	[20.1407547] 	[40503.]	[40580.]
1096	17    	[40567.7]	[18.73526087]	[40540.]	[40592.]
1097	17    	[40601.9]	[18.93911297]	[40580.]	[40639.]
1098	18    	[40624.2]	[15.9298462] 	[40604.]	[40639.]
1099	18    	[40653.4]	[15.46738504]	[40638.]	[40686.]
1100	18    	[40691.9]	[17.14905245]	[40656.]	[40732.]
1101	17    	[40729.8]	[26.9584866] 	[40689.]	[40775.]
1102	18    	[40769.2]	[16.222207]  	[40740.]	[40787.]
1103	18    	[40804.9]	[17.23629891]	[40783.]	[40840.]
1104	19    	[40827.2]	[17.56018223]	[40800.]	[40862.]
1105	19    	[40857.4]	[14.65742133]	[40832.]	[40873.]
1106	18    	[40882.6]	[13.55876101]	[40871.]	[40916.]
1107	18    	[40922.7]	[25.09601562]	[40877.]	[40947.]
1108	18    	[40960.] 	[13.79855065]	[40926.]	[40976.]
1109	17    	[40996.4]	[18.35320136]	[40967.]	[41018.]
1110	16    	[41021.8]	[17.11

1244	19    	[45571.9]	[18.09668478]	[45536.]	[45597.]
1245	19    	[45595.4]	[23.81680079]	[45554.]	[45631.]
1246	17    	[45621.6]	[17.4022987] 	[45592.]	[45640.]
1247	17    	[45650.6]	[13.40298474]	[45637.]	[45683.]
1248	18    	[45692.3]	[16.8940818] 	[45667.]	[45723.]
1249	20    	[45724.3]	[20.22893967]	[45695.]	[45765.]
1250	18    	[45766.5]	[15.31176019]	[45729.]	[45780.]
1251	18    	[45799.2]	[16.63610531]	[45775.]	[45822.]
1252	19    	[45828.] 	[21.73936522]	[45791.]	[45860.]
1253	19    	[45874.] 	[27.61159177]	[45815.]	[45903.]
1254	17    	[45911.8]	[12.02331069]	[45892.]	[45932.]
1255	18    	[45931.] 	[19.38040247]	[45911.]	[45976.]
1256	18    	[45967.9]	[16.6580311] 	[45948.]	[45990.]
1257	18    	[45995.4]	[28.06492473]	[45955.]	[46039.]
1258	18    	[46043.1]	[23.18814352]	[46012.]	[46082.]
1259	20    	[46074.4]	[25.13244914]	[46041.]	[46118.]
1260	17    	[46103.9]	[24.45996729]	[46051.]	[46129.]
1261	16    	[46133.2]	[10.52425769]	[46124.]	[46163.]
1262	18    	[46159.] 	[17.14

1396	17    	[50693.8]	[19.34838495]	[50662.]	[50731.]
1397	17    	[50717.9]	[28.23278236]	[50692.]	[50774.]
1398	19    	[50775.3]	[27.69494539]	[50723.]	[50818.]
1399	20    	[50807.3]	[22.14068653]	[50783.]	[50843.]
1400	19    	[50831.7]	[12.1]       	[50806.]	[50848.]
1401	20    	[50874.1]	[16.39176623]	[50843.]	[50895.]
1402	15    	[50907.2]	[21.12723361]	[50872.]	[50941.]
1403	18    	[50955.6]	[21.51836425]	[50933.]	[50988.]
1404	19    	[50994.5]	[17.07190675]	[50959.]	[51033.]
1405	17    	[51023.] 	[12.83744523]	[50996.]	[51042.]
1406	17    	[51047.6]	[12.15894732]	[51033.]	[51070.]
1407	17    	[51071.8]	[26.53978146]	[51043.]	[51111.]
1408	17    	[51132.5]	[30.44749579]	[51055.]	[51153.]
1409	17    	[51168.] 	[17.96106901]	[51148.]	[51200.]
1410	16    	[51197.] 	[18.12181006]	[51157.]	[51232.]
1411	20    	[51237.] 	[12.59364919]	[51214.]	[51254.]
1412	18    	[51264.2]	[19.00421006]	[51244.]	[51290.]
1413	17    	[51289.6]	[30.79350581]	[51256.]	[51336.]
1414	19    	[51322.4]	[19.34

1548	16    	[55662.8]	[14.1336478] 	[55635.]	[55679.]
1549	18    	[55698.] 	[18.21537812]	[55673.]	[55718.]
1550	18    	[55724.7]	[11.9]       	[55715.]	[55757.]
1551	17    	[55741.8]	[23.53635486]	[55723.]	[55804.]
1552	18    	[55782.8]	[37.61063679]	[55729.]	[55844.]
1553	19    	[55834.3]	[29.96347777]	[55788.]	[55878.]
1554	19    	[55901.6]	[21.5740585] 	[55856.]	[55923.]
1555	18    	[55933.7]	[21.2793327] 	[55908.]	[55974.]
1556	15    	[55965.5]	[11.00227249]	[55934.]	[55974.]
1557	19    	[56009.4]	[15.59615337]	[55978.]	[56021.]
1558	18    	[56034.3]	[21.05730277]	[55992.]	[56068.]
1559	20    	[56080.2]	[18.98315042]	[56041.]	[56114.]
1560	18    	[56116.9]	[11.94529196]	[56087.]	[56133.]
1561	19    	[56164.3]	[10.27667261]	[56155.]	[56184.]
1562	16    	[56199.] 	[18.10524786]	[56182.]	[56221.]
1563	18    	[56236.6]	[19.01157542]	[56221.]	[56271.]
1564	19    	[56269.2]	[11.196428]  	[56243.]	[56282.]
1565	20    	[56303.3]	[12.19877043]	[56280.]	[56321.]
1566	19    	[56326.2]	[9.314

1700	18    	[60661.9]	[17.74514018]	[60639.]	[60686.]
1701	19    	[60690.6]	[5.98665182] 	[60681.]	[60698.]
1702	16    	[60720.6]	[22.18648237]	[60694.]	[60747.]
1703	19    	[60768.9]	[19.56757522]	[60747.]	[60796.]
1704	16    	[60803.4]	[27.61955829]	[60756.]	[60837.]
1705	19    	[60840.] 	[19.19374898]	[60811.]	[60883.]
1706	17    	[60882.1]	[31.526021]  	[60841.]	[60931.]
1707	18    	[60908.1]	[23.43693666]	[60852.]	[60932.]
1708	19    	[60931.7]	[11.55897919]	[60918.]	[60959.]
1709	17    	[60969.4]	[17.03056077]	[60937.]	[60990.]
1710	20    	[60997.4]	[21.59722204]	[60959.]	[61030.]
1711	18    	[61022.6]	[21.34572557]	[60981.]	[61065.]
1712	18    	[61062.1]	[21.91551962]	[61026.]	[61091.]
1713	19    	[61094.7]	[18.14965564]	[61073.]	[61129.]
1714	19    	[61134.6]	[23.80420131]	[61104.]	[61176.]
1715	16    	[61153.8]	[21.04186304]	[61120.]	[61180.]
1716	19    	[61191.5]	[16.62678562]	[61176.]	[61224.]
1717	18    	[61219.6]	[16.99529347]	[61186.]	[61234.]
1718	18    	[61252.5]	[14.84

### Best individual

In [ ]:
solution = Solution(data=np.array(hof[0]), problem=problem)
print(f'Best individual fitness: {int(hof[0].fitness.wvalues[0])}/{problem.rows*problem.cols} ({int(hof[0].fitness.wvalues[0]) / (problem.rows*problem.cols) * 100:.2f}%)')
# print(str(solution))

### Plot evolution

In [ ]:
gen, avg, min_, max_ = log.select("gen", "avg", "min", "max")
plt.plot(gen, avg, label="avg")
plt.plot(gen, min_, label="min")
plt.plot(gen, max_, label="max")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()

In [ ]:
solution.to_file('output/medium.txt')